# Data Augmentation - Conditional Wasserstein GANs - GP

### Dataset: Liver Graft Dataset

This notebook presents the CWGAN-GP model to generate treated Intensity Data from the experimental liver graft dataset.

Notebook Organization:
- Read the dataset
- Treatment, Univariate Analysis and PLS-DA of the dataset
- Setup the CWGAN-GP model and train the model with intensity data
- Generate artificial samples in an artificial dataset and compare them to the experimental data

#### Due to stochasticity, re-running the notebook will get slightly different results. Thus, figures in the paper can be slightly different.


#### Needed Imports

In [ ]:
import json
from time import perf_counter

import numpy as np
import pandas as pd

import scipy.spatial.distance as dist
import scipy.cluster.hierarchy as hier
import scipy.stats as stats
import scipy.spatial

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.colors
from matplotlib import ticker

import seaborn as sns
from collections import namedtuple, Counter

from tqdm import tqdm
from IPython import display as ipythondisplay

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import sklearn.ensemble as skensemble
from sklearn.cross_decomposition import PLSRegression
import sklearn.cluster as skclust
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, cross_validate, GridSearchCV
from sklearn.metrics import (cohen_kappa_score, mean_squared_error, r2_score,
                            adjusted_rand_score, roc_auc_score, roc_curve, auc, f1_score, silhouette_score)

import tensorflow as tf
from keras import backend

import pickle

# Metabolinks package
import metabolinks as mtl
import metabolinks.transformations as transf

# Python files in the repository
import multianalysis as ma
from elips import plot_confidence_ellipse
import gan_evaluation_metrics as gem
import linear_augmentation_functions as laf

In [ ]:
# Import needed functions from GAN_functions
from GAN_functions import gradient_penalty_cwgan
from GAN_functions import critic_loss_wgan
from GAN_functions import generator_loss_wgan

In [ ]:
%matplotlib inline

# Liver Graft Dataset

91 samples belonging to 37 liver graft (for transplant) biopsies, 27 after donor brain death and 10 after cardiac death and 11 Quality Controls. They were obtained in positive ionization mode in a FT-ICR-MS.

Here, we excluded samples analysed during donor surgical phase and quality control, selecting the 74 samples from grafts analysed during cold-phase (after organ retrieval and transportation) and post reperfusion (reconnected graft and stabilised patient haemodynamic). After removing the outlier sample 21, we had 36 samples for each time point.

Class labels: 'Cold-phase' and post 'Reperfusion'.
selected 100 cell extract samples, 50 of them obtained in Complete medium - CC/Complete - and 50 in Defined medium - DC/Defined.

Data acquired by:

- Hrydziuszko, O.; Thamara, M.T.P.R.; Laing, R.; Kirwan, J.; Silva, M.A.; Richards, D.A.; Murphy, N.; Mirza, D.F.; Viant, M.R. Mass Spectrometry Based Metabolomics Comparison of Liver Grafts from Donors after Circulatory Death (DCD) and Donors after Brain Death (DBD) Used in Human Orthotopic Liver Transplantation. PLoS One 2016, 11, 1–16, doi:10.1371/journal.pone.0165884.

- Data available in the paper.

Data matrices retained only features that occur (globally) at least twice in all samples of the dataset (filtering/alignment).

### Reading the data to be analysed and augmented

In [ ]:
base_file = pd.read_excel('LGD_Dataset.XLSX').set_index('StudyID')
base_file

In [ ]:
# Excluding the QC, Donor phase and sample 21
selection = []
for i in range(len(base_file.iloc[:,1])):
    if base_file.iloc[i,1] in [3,4]:
        selection.append(False)
    elif base_file.index[i].startswith('21'):
        selection.append(False)
    else:
        selection.append(True)
base_file = base_file.loc[selection]
base_file

In [ ]:
# Creating the list of 'targets' (labels of samples) of the dataset
labels_temp = list(base_file.iloc[:,1])
labels = []
for i in labels_temp:
    if i == 1:
        labels.append('Cold-phase')
    elif i == 2:
        labels.append('Reperfusion')
labels
pd.Series(labels).value_counts()

In [ ]:
df = base_file.iloc[:,2:].copy()
df = df.replace({0:np.nan})
df_initial = df.copy()

In [ ]:
def characterize_data(dataset, name='dataset', target=None):
    "Returns some basic characteristics about the dataset."

    n_samples, n_feats = dataset.shape

    if target:
        n_classes = len(np.unique(target))
        Samp_Class = len(target)/len(np.unique(target)) # Number of Sample per Class

    avg_feature_value = dataset.values.flatten()[~np.isnan(dataset.values.flatten())].mean() # Mean value in the dataset
    max_feature_value = dataset.values.flatten()[~np.isnan(dataset.values.flatten())].max() # Maximum value in the dataset
    min_feature_value = dataset.values.flatten()[~np.isnan(dataset.values.flatten())].min() # Minimum value in the dataset
    std_feature_value = dataset.values.flatten()[~np.isnan(dataset.values.flatten())].std() # Standard Deviation value
    median_feature_value = np.median(dataset.values.flatten()[~np.isnan(dataset.values.flatten())]) # Median value in the dataset

    if target:
        return {'Dataset': name,
                '# samples': n_samples,
                '# features': n_feats,
                'feature value average (std)': f'{avg_feature_value} ({std_feature_value})',
                'feature value ranges': f'({min_feature_value} - {max_feature_value})',
                'feature value median': median_feature_value,
                '# classes': n_classes,
                'samples / class': Samp_Class,
                } 
    else:
        return {'Dataset': name,
                '# samples': n_samples,
                '# features': n_feats,
                'Feature value average (std)': f'{avg_feature_value} ({std_feature_value})',
                'Feature value ranges': f'({min_feature_value} - {max_feature_value})',
                'Feature value median': median_feature_value,
                }

In [ ]:
data_characteristics = characterize_data(df, target=labels)
data_characteristics

In [ ]:
# See the differences in intensities between samples to see if normalization is required
f, ax = plt.subplots(1,1, figsize=(16,6))

plt.bar(df.index, df.sum(axis=1))
plt.ylabel('Sum of Intensities', fontsize=15)
plt.xlabel('Samples', fontsize=15)
plt.show()

### Data Pre-Treatments

In [ ]:
df  = transf.keep_atleast(df, 2)
df

In [ ]:
# Represents Binary Simplification pre-treatment
def df_to_bool(df):
    "Transforms data into 'binary' matrices."
    return df.mask(df.notnull(), 1).mask(df.isnull(), 0)

# Performs pre-treatment combinations
def compute_transf(df, norm_ref=None, lamb=None):
    "Computes combinations of pre-treatments and BinSim and returns after treatment datasets in a dict."
    
    data = df.copy()
    
    # Imputation of Missing Values
    imputed = transf.fillna_frac_min_feature(data.T, fraction=0.2).T

    # Normalization
    if norm_ref is not None:
        # Normalization by a reference feature
        norm = transf.normalize_ref_feature(imputed, norm_ref, remove=True)
    else:
        # Normalization by the total sum of intensities
        #norm = transf.normalize_sum(imputed)
        # Normalization by PQN
        norm = transf.normalize_PQN(imputed, ref_sample='mean')
    
    # Pareto Scaling and Generalized Logarithmic Transformation
    P = transf.pareto_scale(imputed)
    NP = transf.pareto_scale(norm)
    NGP = transf.pareto_scale(transf.glog(norm, lamb=lamb))
    GP = transf.pareto_scale(transf.glog(imputed, lamb=lamb))
    
    # Store results
    dataset = {}
    dataset['data'] = df

    dataset['BinSim'] = df_to_bool(data)
    dataset['Ionly'] = imputed
    dataset['P'] = P
    dataset['NP'] = NP
    dataset['GP'] = GP
    dataset['NGP'] = NGP

    return dataset

In [ ]:
df = df.replace({0:np.nan})
p_df = compute_transf(df, norm_ref=None, lamb=None)

In [ ]:
target = labels

#### Colors for plots to ensure consistency

In [ ]:
colours = sns.color_palette('Set1', 6)

ordered_labels = set(labels)

label_colors = {lbl: c for lbl, c in zip(ordered_labels, colours)}
sample_colors = [label_colors[lbl] for lbl in labels]

sns.palplot(label_colors.values())
new_ticks = plt.xticks(range(len(ordered_labels)), ordered_labels)

### Calculating Silhouette Coefficient

The Silhouette coefficient was calculated (with scikit learn's `silhouette score`) using the class labels as the clusters in order to give a measure of how overlapped the biological classes are between the different datasets.

The coefficient was calculated using all features of the dataset after pre-treatment and using the PCA Scores of the 2 first Principal Components.

In [ ]:
# Silhoutte Coefficient based on all the features of the dataset after pre-treatment
print('Calculation based on all the features of the dataset after pre-treatment')
print('Silhouette Coefficient:', silhouette_score(p_df['NGP'], labels))

In [ ]:
# Silhoutte Coefficient based on the PCA Scores of the samples on the 2 first Principal Components
principaldf, var = ma.compute_df_with_PCs(p_df['NGP'], n_components=2, whiten=True,
                                          labels=labels, return_var_ratios=True)
s_score = silhouette_score(principaldf.iloc[:,:2], labels)
print('Calculation based on the PCA Scores of the samples on the 2 first Principal Components')
print('Silhouette Coefficient:', s_score)

## Unsupervised Analysis

#### PCA

In [ ]:
def plot_PCA(principaldf, label_colors, components=(1,2), title="PCA", ax=None):
    "Plot the projection of samples in the 2 main components of a PCA model."
    
    if ax is None:
        ax = plt.gca()
    
    loc_c1, loc_c2 = [c - 1 for c in components]
    col_c1_name, col_c2_name = principaldf.columns[[loc_c1, loc_c2]]
    
    #ax.axis('equal')
    ax.set_xlabel(f'{col_c1_name}')
    ax.set_ylabel(f'{col_c2_name}')

    unique_labels = principaldf['Label'].unique()

    for lbl in unique_labels:
        subset = principaldf[principaldf['Label']==lbl]
        ax.scatter(subset[col_c1_name],
                   subset[col_c2_name],
                   s=50, color=label_colors[lbl], label=lbl)

    #ax.legend(framealpha=1)
    ax.set_title(title, fontsize=15)

def plot_ellipses_PCA(principaldf, label_colors, components=(1,2),ax=None, q=None, nstd=2):
    "Plot confidence ellipses of a class' samples based on their projection in the 2 main components of a PCA model."
    
    if ax is None:
        ax = plt.gca()
    
    loc_c1, loc_c2 = [c - 1 for c in components]
    points = principaldf.iloc[:, [loc_c1, loc_c2]]
    
    #ax.axis('equal')

    unique_labels = principaldf['Label'].unique()

    for lbl in unique_labels:
        subset_points = points[principaldf['Label']==lbl]
        plot_confidence_ellipse(subset_points, q, nstd, ax=ax, ec=label_colors[lbl], fc='none')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8,8))

tf_fs = transf.FeatureScaler(method='standard')
df = p_df['NGP']

ax.axis('equal')
principaldf, var = ma.compute_df_with_PCs(df, n_components=2, whiten=True, labels=labels, return_var_ratios=True)

lcolors = label_colors

plot_PCA(principaldf, lcolors, components=(1,2), title='', ax=ax)
plot_ellipses_PCA(principaldf, lcolors, components=(1,2),ax=ax, q=0.95)

ax.set_xlabel(f'PC 1 ({var[0] * 100:.1f} %)')
ax.set_ylabel(f'PC 2 ({var[1] * 100:.1f} %)')

plt.legend()
plt.show()

#### HCA

In [ ]:
def perform_HCA(df, metric='euclidean', method='average'):
    "Performs Hierarchical Clustering Analysis of a data set with chosen linkage method and distance metric."
    
    distances = dist.pdist(df, metric=metric)
    
    # method is one of
    # ward, average, centroid, single, complete, weighted, median
    Z = hier.linkage(distances, method=method)

    # Cophenetic Correlation Coefficient
    # (see how the clustering - from hier.linkage - preserves the original distances)
    coph = hier.cophenet(Z, distances)
    # Baker's gamma
    mr = ma.mergerank(Z)
    bg = mr[mr!=0]

    return {'Z': Z, 'distances': distances, 'coph': coph, 'merge_rank': mr, "Baker's Gamma": bg}

In [ ]:
HCA_all = {}
for treat in 'Ionly', 'P', 'NP', 'GP', 'NGP', 'BinSim':
    print(f'Performing HCA with treatment {treat}', end=' ...')
    metric = 'jaccard' if treat == 'BinSim' else 'euclidean'
    HCA_all[treat] = perform_HCA(p_df[treat], metric=metric, method='ward')
    print('done!')

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def color_list_to_matrix_and_cmap(colors, ind, axis=0):
        if any(issubclass(type(x), list) for x in colors):
            all_colors = set(itertools.chain(*colors))
            n = len(colors)
            m = len(colors[0])
        else:
            all_colors = set(colors)
            n = 1
            m = len(colors)
            colors = [colors]
        color_to_value = dict((col, i) for i, col in enumerate(all_colors))

        matrix = np.array([color_to_value[c]
                           for color in colors for c in color])

        matrix = matrix.reshape((n, m))
        matrix = matrix[:, ind]
        if axis == 0:
            # row-side:
            matrix = matrix.T

        cmap = mpl.colors.ListedColormap(all_colors)
        return matrix, cmap

def plot_dendogram2(Z, leaf_names, label_colors, title='', ax=None, no_labels=False, labelsize=12, **kwargs):
    if ax is None:
        ax = plt.gca()
    hier.dendrogram(Z, labels=leaf_names, leaf_font_size=10, above_threshold_color='0.2', orientation='left',
                    ax=ax, **kwargs)
    #Coloring labels
    #ax.set_ylabel('Distance (AU)')
    ax.set_xlabel('Distance (AU)')
    ax.set_title(title, fontsize = 15)
    
    #ax.tick_params(axis='x', which='major', pad=12)
    ax.tick_params(axis='y', which='major', labelsize=labelsize, pad=12)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    #xlbls = ax.get_xmajorticklabels()
    xlbls = ax.get_ymajorticklabels()
    rectimage = []
    for lbl in xlbls:
        col = label_colors[lbl.get_text()]
        lbl.set_color(col)
        #lbl.set_fontweight('bold')
        if no_labels:
            lbl.set_color('w')
        rectimage.append(col)

    cols, cmap = color_list_to_matrix_and_cmap(rectimage, range(len(rectimage)), axis=0)

    axins = inset_axes(ax, width="5%", height="100%",
                   bbox_to_anchor=(1, 0, 1, 1),
                   bbox_transform=ax.transAxes, loc=3, borderpad=0)

    axins.pcolor(cols, cmap=cmap, edgecolors='w', linewidths=1)
    axins.axis('off')

In [ ]:
with sns.axes_style("white"):
    f, axs = plt.subplots(1, 4, figsize=(14, 8), constrained_layout=True)
    
    for treatment, ax in zip(('P', 'NP', 'GP', 'NGP'), axs.ravel()):
        plot_dendogram2(HCA_all[treatment]['Z'], 
                       labels, ax=ax,
                       label_colors=label_colors,
                       title=treatment, color_threshold=0)

    st = f.suptitle(f"1/5 Min Imputation", fontsize=16)
    plt.show()

In [ ]:
with sns.axes_style("white"):
    f, axs = plt.subplots(1, 1, figsize=(4, 8), constrained_layout=True)
    
    treatment = 'BinSim'
    ax = axs
    plot_dendogram2(HCA_all[treatment]['Z'], 
                   labels, ax=ax,
                   label_colors=label_colors,
                   title=treatment, color_threshold=0)

    #st = f.suptitle(f"1/5 Min Imputation", fontsize=16)
    plt.show()

In [ ]:
def compute_clustering_metrics(res_dict, labels):
    """Fill dict with clustering performance metrics."""
    
    discrim = ma.dist_discrim(res_dict['Z'], labels, # all samples have the same order
                              method = 'average')
    res_dict['Average discrim dist'] = discrim[0]
    correct = np.array(list(discrim[1].values()))
    
    classes = pd.unique(labels)
    res_dict['% correct clustering'] = (100/len(classes)) * len(correct[correct>0])

    # Correct First Cluster Percentage
    res_dict['% correct 1st clustering'] = 100 * ma.correct_1stcluster_fraction(res_dict['Z'],labels)

In [ ]:
for name, res_dict in HCA_all.items():
    compute_clustering_metrics(res_dict, labels)

# Build table - summary of results
clust_performance = {}

for metric in ('Average discrim dist', '% correct clustering', '% correct 1st clustering'):
    clust_performance[metric] = {d: HCA_all[d][metric] for d in HCA_all}
clust_performance = pd.DataFrame(clust_performance, index=HCA_all)
clust_performance

#### K-Means Clustering

In [ ]:
def perform_KMeans(dataset, treatment, iter_num=150, best_fraction=0.1):
    "Perform K-means Clustering Analysis and calculate discrimination evaluation metrics."
    
    sample_labels = labels
    n_classes = len(pd.unique(sample_labels))
    
    df = dataset[treatment]
    
    discrim = ma.Kmeans_discrim(df, sample_labels,
                                method='average', 
                                iter_num=iter_num,
                                best_fraction=best_fraction)

    # Lists for the results of the best k-means clustering
    average = []
    correct = []
    rand = []
    
    for j in discrim:
        global_disc_dist, disc_dists, rand_index, SSE = discrim[j]
        
        # Average of discrimination distances
        average.append(global_disc_dist) 
        
        # Correct Clustering Percentages
        all_correct = np.array(list(disc_dists.values()))
        correct.append(len(all_correct[all_correct>0]))
        
        # Adjusted Rand Index
        rand.append(rand_index) 
    
    return{'dataset': dataset,
           'treatment': treatment,
           'Discrimination Distance': np.median(average),
           '% correct clusters':np.median(correct)*100/n_classes,
           'Rand Index': np.median(rand)}

In [ ]:
iter_num=15

KMeans_all = []

for treatment in ('Ionly', 'P', 'NP', 'GP', 'NGP', 'BinSim'):
    print(f'performing KMeans with treatment {treatment}' , end=' ...')
    KMeans_all.append(perform_KMeans(p_df, treatment, iter_num=iter_num))
    print('done!')        

In [ ]:
KMeans_all = pd.DataFrame(KMeans_all).iloc[:,1:]
KMeans_all

### Supervised Analysis

#### RF

In [ ]:
target = labels
GENERATE = True
np.random.seed(5)
if GENERATE:

    # Vector with values for the parameter n_estimators
    # Models will be built from 10 to 200 trees in 5 tree intervals
    top_tree_in_grid=200
    values = {'n_estimators': range(10,top_tree_in_grid,5)}

    rf = skensemble.RandomForestClassifier(n_estimators=200)
    clf = GridSearchCV(rf, values, cv=5)

    # For each dataset,building the Random Forest models with the different number of trees
    # and storing the predictive accuracy
    RF_optim = {}
    for treatment in ('Ionly','P', 'NP', 'GP', 'NGP', 'BinSim'):
        print('Fitting to pre-treatment', treatment, '...', end=' ')
        rfname = treatment
        RF_optim[rfname] = {'treatment':treatment}
        clf.fit(p_df[treatment], target)

        RF_optim[rfname]['scores'] = list(clf.cv_results_['mean_test_score'])
        RF_optim[rfname]['n_trees'] = list(clf.cv_results_['param_n_estimators'])

        print('Done!')

In [ ]:
# Plotting the results and adjusting parameters of the plot

def plot_RF_otimization_ntrees(RF_optim, ax=None, ylabel='', title='', ylim=(30,101)):
    p7 = sns.color_palette('tab20', 9)
    to_plot = [optim for key, optim in RF_optim.items()]
    treatments = ('Ionly', 'P', 'NP', 'GP', 'NGP', 'BinSim')
    if ax is None:
        ax = plt.gca()
    for treatment, color in zip(treatments, p7):
        for optim in to_plot:
            if optim['treatment'] == treatment:
                break
        ax.plot(optim['n_trees'], [s*100 for s in optim['scores']], label=treatment, color=color)
    ax.set(ylabel=ylabel, xlabel='Number of Trees', ylim=ylim, title=title)
    ax.legend()

with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize=(6,6), constrained_layout=True)
        
        plot_RF_otimization_ntrees(RF_optim, ax=ax,
                                   ylabel='Random Forest CV Mean Accuracy (%)',
                                   title='')

        f.suptitle('Optimization of the number of trees')

        plt.show()

In [ ]:
# RF_model_CV - RF application and result extraction.
def RF_model_CV(df, y, iter_num=1, n_fold=5, n_trees=200):
    """Performs stratified k-fold cross validation on Random Forest classifier of a dataset n times giving its accuracy and ordered
    most important features.

       Parameters are estimated by stratified k-fold cross-validation. Iteration changes the random sampling of the k-folds for
    cross-validation.
       Feature importance in the Random Forest models is calculated by the Gini Importance (feature_importances_) of scikit-learn.

       df: Pandas DataFrame.
       y: the target array (following scikit learn conventions)
       iter_num: int (default - 1); number of iterations for CV.
       n_fold: int (default - 5); number of groups to divide dataset in for stratified k-fold cross-validation
            (max n_fold = minimum number of samples belonging to one group).
       n_trees: int (default - 200); number of trees in each Random Forest.

       Returns: (scores, import_features); 
            scores: list of the scores/accuracies of k-fold cross-validation of the random forests
                (one score for each iteration and each group)
            import_features: list of tuples (index number of feature, feature importance, feature name)
                ordered by decreasing feature importance.
    """

    nfeats = df.shape[1]

    # Setting up variables for result storing
    imp_feat = np.zeros((iter_num * n_fold, nfeats))
    accuracy_scores = []
    f1_scores = []
    f = 0

    # Number of times Random Forest cross-validation is made
    # with `n_fold` randomly generated folds.
    for _ in range(iter_num):
        # Use stratified n_fold cross validation
        kf = StratifiedKFold(n_fold, shuffle=True)
        CV_accuracy_scores = []
        CV_f1_scores = []
        # Fit and evaluate a Random Forest model for each fold
        for train_index, test_index in kf.split(df, y):
            # Random Forest setup and fit
            rf = skensemble.RandomForestClassifier(n_estimators=n_trees)
            X_train, X_test = df.iloc[train_index, :], df.iloc[test_index, :]
            y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
            rf.fit(X_train, y_train)

            # Compute performance and important features
            CV_accuracy_scores.append(rf.score(X_test, y_test)) # Predictive Accuracy
            #print(y_test, rf.predict(X_test))
            CV_f1_scores.append(f1_score(y_true=y_test, y_pred=rf.predict(X_test), average='weighted')) # F1-Scores
            #print(f1_score(y_true=y_test, y_pred=rf.predict(X_test), pos_label='Parkinson'))
            imp_feat[f, :] = rf.feature_importances_ # Importance of each feature
            f = f + 1

        # Average Predictive Accuracy in this iteration
        accuracy_scores.append(np.mean(CV_accuracy_scores))
        f1_scores.append(np.mean(CV_f1_scores))

    # Collect and order all important features values from each Random Forest
    imp_feat_sum = imp_feat.sum(axis=0) / (iter_num * n_fold)
    sorted_imp_feat = sorted(enumerate(imp_feat_sum), key=lambda x: x[1], reverse=True)

    # locs are sufficient as a reference to features
    #imp_feat_tuples = [(loc, importance) for loc, importance in sorted_imp_feat]
    
    if iter_num == 1:
        return {'accuracy': accuracy_scores[0], 'f1-score': f1_scores[0], 'important_features': sorted_imp_feat}
    else:
        return {'accuracy': accuracy_scores, 'f1-score': f1_scores, 'important_features': sorted_imp_feat}

In [ ]:
iter_num=20

RF_all = {}

# Application of the Random Forests for each differently-treated dataset
for treatment in ('Ionly', 'P', 'NP', 'GP', 'NGP', 'BinSim'):
    print(f'Fitting random forest with treatment {treatment}', end=' ...')
    rfname = treatment
    RF_all[rfname] = {'treatment':treatment}
    n_fold = 5

    fit = RF_model_CV(p_df[treatment], target, iter_num=iter_num, n_fold=n_fold, n_trees=100)
    RF_all[rfname].update(fit)

    print(f'done')    

In [ ]:
# Accuracy across the iterations
accuracies = pd.DataFrame({name: RF_all[name]['accuracy'] for name in RF_all})
# F1-Score across the iterations
f1scores = pd.DataFrame({name: RF_all[name]['f1-score'] for name in RF_all})

In [ ]:
accuracy_stats = pd.DataFrame({'Average accuracy': accuracies.mean(axis=0),
                               'STD': accuracies.std(axis=0)})
accuracy_stats = accuracy_stats.assign(treatment=[RF_all[name]['treatment'] for name in RF_all])
accuracy_stats

In [ ]:
f1scores_stats = pd.DataFrame({'Average f1-scores': f1scores.mean(axis=0),
                               'STD': f1scores.std(axis=0)})
f1scores_stats = f1scores_stats.assign(treatment=[RF_all[name]['treatment'] for name in RF_all])
f1scores_stats

#### PLS-DA

In [ ]:
%%capture --no-stdout
# above is to supress PLS warnings

max_comp=20
np.random.seed(9125)
# Store Results
PLS_optim = {}

# Build and extract metrics from models build with different number of components by using the optim_PLS function.
for treatment in ('Ionly', 'P', 'NP', 'GP', 'NGP', 'BinSim'):
    print(f'Fitting PLS-DA model with treatment {treatment}', end=' ...')
    plsdaname = treatment
    PLS_optim[plsdaname] = {'treatment':treatment}
    n_fold = 5
    optim = ma.optim_PLSDA_n_components(p_df[treatment], target,
                                        max_comp=max_comp, n_fold=n_fold).CVscores
    PLS_optim[plsdaname]['CV_scores'] = optim
    print(f'done')

In [ ]:
PLS_optim['NGP']['CV_scores']

In [ ]:
# Plotting the results and adjusting plot parameters
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize = (8,8))
        for name, data in PLS_optim.items():

            # Negative Grapevine Dataset
            ax.plot(range(1, len(data['CV_scores']) + 1), data['CV_scores'],
                     label=data['treatment'])
            ax.set(xlabel='Number of Components',
                    ylabel='PLS Score (1 - PRESS/SS)',
                    title='Negative Mode Grapevine Dataset')
            ax.legend()
            ax.set_ylim([-1, 1])

        plt.tight_layout()
        plt.show()

In [ ]:
def PLSDA_model_CV(df, labels, n_comp=10,
                   n_fold=5,
                   iter_num=1,
                   encode2as1vector=True,
                   feat_type='Coef'):
    
    """Perform PLS-DA with n-fold cross-validation.

    df: pandas DataFrame; includes X equivalent in PLS-DA (training vectors).
    labels: target labels.
    n_comp: integer; number of components to use in PLS-DA.
    n_fold: int (default: 5); number of groups to divide dataset in for k-fold cross-validation
        (NOTE: max n_fold can not exceed minimum number of samples per class).
    iter_num: int (default: 1); number of iterations that cross validation is repeated.
    feat_type: string (default: 'Coef'); types of feature importance metrics to use; accepted: {'VIP', 'Coef', 'Weights'}.

    Returns: (accuracy, n-fold score, r2score, import_features);
        accuracy: list of accuracy values in group selection
        n-fold score : n-fold cross-validation score
        r2score: r2 score of the model
        import_features: list of tuples (index number of feature, feature importance, feature name)
            ordered by decreasing feature importance.
    """
    # Setting up lists and matrices to store results
    CVR2 = []
    accuracies = []
    f1scores = []
    Imp_Feat = np.zeros((iter_num * n_fold, df.shape[1]))
    f = 0

    unique_labels = list(pd.unique(labels))

    is1vector = len(unique_labels) == 2 and encode2as1vector

    matrix = ma._generate_y_PLSDA(labels, unique_labels, is1vector)

    if is1vector:
        # keep a copy to use later
        target1D = matrix.copy()

    # Number of iterations equal to iter_num
    for i in range(iter_num):
        # use startified n-fold cross-validation with shuffling
        kf = StratifiedKFold(n_fold, shuffle=True)
        
        # Setting up storing variables for n-fold cross-validation
        nright = 0
        cvr2 = []
        cvf1scores = []

        # Iterate through cross-validation procedure
        for train_index, test_index in kf.split(df, labels):
            plsda = PLSRegression(n_components=n_comp, scale=False)
            X_train, X_test = df.iloc[train_index, :], df.iloc[test_index, :]
            if not is1vector:
                y_train = matrix.iloc[train_index, :].copy()
                y_test = matrix.iloc[test_index, :].copy()

            else:
                y_train, y_test = target1D[train_index], target1D[test_index]
                correct = target1D[test_index]

            # Fit PLS model
            plsda.fit(X=X_train, Y=y_train)

            # Obtain results with the test group
            y_pred = plsda.predict(X_test)
            cvr2.append(r2_score(y_test, y_pred))
            
            if not is1vector:
                rounded_pred = y_pred.copy()
                for i in range(len(y_pred)):
                    
                    for l in range(len(y_pred[i])):
                        if y_pred[i,l] > 0.5:
                            rounded_pred[i, l] = 1
                        else:
                            rounded_pred[i, l] = 0

                cvf1scores.append(f1_score(y_test, rounded_pred, average='weighted'))
    

            if not is1vector:
                for i in range(len(y_pred)):
                    if list(y_test.iloc[i, :]).index(max(y_test.iloc[i, :])) == np.argmax(
                        y_pred[i]
                    ):
                        nright += 1  # Correct prediction
            else:
                
                rounded = np.round(y_pred)
                for p in range(len(y_pred)):
                    if rounded[p] >= 1:
                        pred = 1
                        rounded[p] = 1
                    else:
                        pred = 0
                        rounded[p] = 0
                    if pred == correct[p]:
                        nright += 1  # Correct prediction

                # Calculate accuracy for this iteration
                cvf1scores.append(f1_score(correct, rounded, pos_label=0))

            # Calculate important features (3 different methods to choose from)
            if feat_type == 'VIP':
                Imp_Feat[f, :] = ma._calculate_vips(plsda)
            elif feat_type == 'Coef':
                Imp_Feat[f, :] = abs(plsda.coef_).sum(axis=1)
            elif feat_type == 'Weights':
                Imp_Feat[f, :] = abs(plsda.x_weights_).sum(axis=1)
            else:
                raise ValueError(
                    'Type not Recognized. Types accepted: "VIP", "Coef", "Weights"'
                )

            f += 1

        # Calculate the accuracy of the group predicted and storing score results
        accuracies.append(nright / len(labels))
        CVR2.append(np.mean(cvr2))
        f1scores.append(np.mean(cvf1scores))

    # Join and sort all important features values from each cross validation group and iteration.
    Imp_sum = Imp_Feat.sum(axis=0) / (iter_num * n_fold)
    imp_features = sorted(enumerate(Imp_sum), key=lambda x: x[1], reverse=True)
    if iter_num == 1:
        return {'accuracy': accuracies[0], 'Q2': CVR2[0], 'f1-score': f1scores[0], 'important_features': imp_features}
    else:
        return {'accuracy': accuracies, 'Q2': CVR2, 'f1-score': f1scores, 'important_features': imp_features}

In [ ]:
%%capture --no-stdout

PLSDA_all = {}

iter_num=20

for treatment in ('Ionly', 'P', 'NP', 'GP', 'NGP', 'BinSim'):
    print(f'Fitting a PLS-DA model with treatment {treatment}', end=' ...')
    plsdaname = treatment
    PLSDA_all[plsdaname] = {'treatment':treatment}
    n_comp = 4
    n_fold = 5
    fit = PLSDA_model_CV(p_df[treatment], target,
                            n_comp=n_comp, n_fold=n_fold,
                            iter_num=iter_num,
                            feat_type='VIP')
    PLSDA_all[plsdaname].update(fit)
    print(f'done')     

In [ ]:
accuracies = pd.DataFrame({name: PLSDA_all[name]['accuracy'] for name in PLSDA_all})
f1scores = pd.DataFrame({name: PLSDA_all[name]['f1-score'] for name in PLSDA_all})

In [ ]:
accuracy_stats = pd.DataFrame({'Average accuracy': accuracies.mean(axis=0),
                               'STD': accuracies.std(axis=0)})
accuracy_stats = accuracy_stats.assign(treatment=[PLSDA_all[name]['treatment'] for name in PLSDA_all])
accuracy_stats

In [ ]:
f1scores_stats = pd.DataFrame({'Average f1scores': f1scores.mean(axis=0),
                               'STD': f1scores.std(axis=0)})
f1scores_stats = f1scores_stats.assign(treatment=[PLSDA_all[name]['treatment'] for name in PLSDA_all])
f1scores_stats

In [ ]:
n_components = 4 # Nº of componentes

model, scores = ma.fit_PLSDA_model(p_df['NGP'], target,
                                n_comp=n_components, scale=False, # Only true if scaling was not done earlier
                                encode2as1vector=True,
                                lv_prefix='LV ', label_name='Label')

lcolors = label_colors

with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        fig, ax = plt.subplots(1,1, figsize=(6,6)) # Set up fig size
        plot_PCA(scores, lcolors, title="PLS Projection", ax=ax,
                components=(1,2)) # Select components to see
        plt.title('PLS Projection', fontsize=20) # Title
        plt.legend(loc='upper right', ncol=1, fontsize=15)  # Legend           
        plt.tight_layout()
        plt.show()
        

Data pre-treatment

In [ ]:
def basic_feat_filtering(file, target=None, filt_method='total_samples', filt_kw=2,
                  extra_filt=None, extra_filt_kw='Formula', extra_filt_data=None):
    "Performs feature filtering in 2 steps."
    
    # Filtering based on the sampels each feature appears in
    if filt_method == 'total_samples': # Filter features based on the times (filt_kw) they appear in the dataset
        # Minimum can be a percentage if it is a value between 0 and 1!
        data_filt = transf.keep_atleast(file, minimum=filt_kw)
    elif filt_method == 'class_samples': # Features retained if they appear filt_kw times in the samples of at least one class
        # Minimum can be a percentage if it is a value between 0 and 1!
        data_filt = transf.keep_atleast(file, minimum=filt_kw, y=np.array(target))
    elif filt_method == None: # No filtering
        data_filt = file.copy()
    else:
        raise ValueError('Feature Filtering strategy not accepted/implemented in function. Implement if new strategy.')
        
    # Extra filtering based if the features are annotated
    if extra_filt == 'Formula': # Filter features based if they have a formula on the dataset
        idxs_to_keep = [i for i in data_filt.index if type(extra_filt_data.loc[i, 'Formula']) == str]
        data_filt = data_filt.loc[idxs_to_keep]
    elif extra_filt == 'Name': # Filter features based if they have an annotated name on the dataset
        idxs_to_keep = [i for i in data_filt.index if type(extra_filt_data.loc[i, 'Name']) == str]
        data_filt = data_filt.loc[idxs_to_keep]
    elif extra_filt == None: # No extra filtering
        data_filt = data_filt.copy()
    else:
        raise ValueError('Feature Filtering strategy not accepted/implemented in function. Implement if new strategy.')
    
    return data_filt

In [ ]:
filt_sample_data = basic_feat_filtering(df_initial, target, filt_method='total_samples', filt_kw=2).T
imputed = transf.fillna_frac_min_feature(filt_sample_data, fraction=0.2).T
normalized = transf.normalize_sum(imputed)
real_samples_all = transf.pareto_scale(transf.glog(normalized, lamb=None))

#### Univariate Analysis

In [ ]:
uni_results = ma.compute_FC_pvalues_2groups(normalized, real_samples_all,
                               labels=target,
                               equal_var=True,
                               alpha=0.05, useMW=False)
uni_results

In [ ]:
pvalue = 0.0001
log2FC = 1

b = uni_results[uni_results['FDR adjusted p-value'] < pvalue]
uni_results_filt = b[abs(b['log2FC']) > log2FC]
uni_results_filt

#### Linear Augmentation / Interpolation of the experimental data and corresponding sample labels/classes to use as training data for the GAN network.

Pre-treatment on the linearly augmented data.

In [ ]:
start = perf_counter()
data, lbls = laf.artificial_dataset_generator(imputed, labels=labels,
                                    max_new_samples_per_label=512, binary=False, rnd=list(np.linspace(0.15,0.9,6)), 
                                    binary_rnd_state=None, rnd_state=45236)

data_N = transf.normalize_sum(data)
data_treated = transf.pareto_scale(transf.glog(data_N, lamb=None))

end = perf_counter()
print(f'Simple augmentation of data done! Took {(end - start):.3f} s')

### Inputs for GAN model

In [ ]:
# Get distribution of intensity values of the dataset
hist = np.histogram(real_samples_all.values.flatten(), bins=100)
input_realdata_dist = stats.rv_histogram(hist)

Set up colours for each of the classes. Generated samples will have the corresponding label with '- GAN' after.

In [ ]:
# Colors to use in plots
colours2 = sns.color_palette('tab20', 4)#[:6]

ordered_labels_test = []
for i in ['Cold-phase', 'Reperfusion']:
    ordered_labels_test.extend([i, i + ' - GAN'])
label_colors_test = {lbl: c for lbl, c in zip(ordered_labels_test, colours2)}

sns.palplot(label_colors_test.values())
new_ticks_test = plt.xticks(range(len(ordered_labels_test)), ordered_labels_test)

## Conditional Wasserstein GAN - GP model

This model construction was made by joining WGAN-GP models with Conditional GAN models. WGAN-GP models were originally made according to / originally based in https://keras.io/examples/generative/wgan_gp/#wasserstein-gan-wgan-with-gradient-penalty-gp and Conditional GAN models - https://machinelearningmastery.com/how-to-develop-a-conditional-generative-adversarial-network-from-scratch/ (generator and discriminator model) and https://keras.io/examples/generative/conditional_gan/ without using OOP (loss functions and training/training steps).

Functions for the generator and critic (discriminator) models

In [ ]:
def generator_model(len_input, len_output, n_hidden_nodes, n_labels): 
    "Make the generator model of CWGAN-GP."

    data_input = tf.keras.Input(shape=(len_input,), name='data') # Take intensity input
    label_input = tf.keras.Input(shape=(1,), name='label') # Take Label Input
    
    # Treat label input to concatenate to intensity data after
    label_m = tf.keras.layers.Embedding(n_labels, 30, input_length=1)(label_input)
    label_m = tf.keras.layers.Dense(256, activation='linear', use_bias=True)(label_m)
    #label_m = tf.keras.layers.Reshape((len_input,1,))(label_m)
    label_m2 = tf.keras.layers.Reshape((256,))(label_m)

    joined_data = tf.keras.layers.Concatenate()([data_input, label_m2]) # Concatenate intensity and label data
    # Hidden Dense Layer and Normalization
    joined_data = tf.keras.layers.Dense(n_hidden_nodes, activation=tf.nn.leaky_relu, use_bias=True)(joined_data)
    joined_data = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, use_bias=True)(joined_data)
    joined_data = tf.keras.layers.BatchNormalization()(joined_data)
    
    # Output - number of features of sample to make
    output = tf.keras.layers.Dense(len_output, activation='linear', use_bias=True)(joined_data)
    
    generator = tf.keras.Model(inputs=[data_input, label_input], outputs=output)
    
    return generator


def critic_model(len_input, n_hidden_nodes, n_labels):
    "Make the critic model of CWGAN-GP."
    
    label_input = tf.keras.Input(shape=(1,)) # Take intensity input
    data_input = tf.keras.Input(shape=(len_input,)) # Take Label Input

    # Treat label input to concatenate to intensity data after
    label_m = tf.keras.layers.Embedding(n_labels, 30, input_length=1)(label_input)
    label_m = tf.keras.layers.Dense(256, activation='linear', use_bias=True)(label_m)
    #label_m = tf.keras.layers.Reshape((len_input,1,))(label_m)
    label_m = tf.keras.layers.Reshape((256,))(label_m)

    joined_data = tf.keras.layers.Concatenate()([data_input, label_m]) # Concatenate intensity and label data
    # Hidden Dense Layer (Normalization worsened results here)
    joined_data = tf.keras.layers.Dense(n_hidden_nodes, activation=tf.nn.leaky_relu, use_bias=True)(joined_data)
    joined_data = tf.keras.layers.Dense(128, activation=tf.nn.leaky_relu, use_bias=True)(joined_data)
    joined_data = tf.keras.layers.Dense(256, activation=tf.nn.leaky_relu, use_bias=True)(joined_data)
    #joined_data = tf.keras.layers.BatchNormalization()(joined_data)

    # Output Layer - 1 node for critic decision
    output = tf.keras.layers.Dense(1, activation='linear', use_bias=True)(joined_data)
    
    critic = tf.keras.Model(inputs=[data_input, label_input], outputs=output)

    return critic

In [ ]:
def generate_predictions(model, num_examples_to_generate, len_input, input_dist, uni_lbls):
    "Generate sample predictions based on a Generator model."
    
    test_input =  tf.constant(input_dist.rvs(size=len_input*num_examples_to_generate), shape=[
        num_examples_to_generate,len_input]) 
    
    if len(uni_lbls) < 3:
        test_labels = tf.constant([1.0]*(num_examples_to_generate//2) + [0.0]*(num_examples_to_generate//2), 
                                  shape=(num_examples_to_generate,1))
    else:
        test_labels = []
        for i in range(len(uni_lbls)):
            test_labels.extend([i]*(num_examples_to_generate//len(uni_lbls)))
        test_labels = np.array(pd.get_dummies(test_labels))
        #np.array(pd.get_dummies([i for i in range(len(uni_lbls))]*(num_examples_to_generate//len(uni_lbls))))
    predictions = model([test_input, test_labels], training=False) # `training` is set to False.
    return predictions

In [ ]:
def training_montage(train_data_o, train_lbls, test_data, test_lbls,
                     epochs, generator, critic, generator_optimizer, critic_optimizer, input_dist,
                    batch_size, grad_pen_weight=10, k_cov_den=50, k_crossLID=15, random_seed=145,
                    n_generated_samples=96):
    """Train a generator and critic of CWGAN-GP.
    
       Receives training data and respective class labels (train_data_o and train_lbls) and trains a generator and a critic
        model (generator, critic) over a number of epochs (epochs) with a set batch size (batch_size) with the respective 
        optimizers and learning rate (generator_optimizer, critic_optimizer). Gradient Penalty is calculated with
        grad_pen_weight as the weight of the penalty.
       The functions returns at time intervals three graphs to evaluate the progression of the models (Loss plots,
        coverage, density, crossLID and correct first cluster plots and PCA plot with generated and test data). To this
        end, samples need to be generated requiring the distribution to sample the initial input values from (input_dist),
        and test data and respective labels has to be given (test_data and test_lbls). Finally the number of neighbors to
        consider for coverage/density and crossLID calculation is also needed (k_cov_den, k_crossLID).
    
       train_data_o: Pandas DataFrame with training data;
       train_lbls: List with training data class labels;
       test_data: Pandas DataFrame with test data to evaluate the model;
       test_lbls: List with test data class labels to evaluate the model;
       epochs: Int value with the number of epochs to train the model;
       generator: tensorflow keras.engine.functional.Functional model for the generator;
       critic: tensorflow keras.engine.functional.Functional model for the critic;
       generator_optimizer: tensorflow keras optimizer (with learning rate) for generator;
       critic_optimizer: tensorflow keras optimizer (with learning rate) for critic;
       input_dist: scipy.stats._continuous_distns.rv_histogram object - distribution to sample input values for generator;
       batch_size: int value with size of batch for model training;
       grad_pen_weight: int value (default 10) for penalty weight in gradient penalty calculation;
       k_cov_den: int value (default 50) for number of neighbors to consider for coverage and density calculation in
       generated samples evaluation;
       k_crossLID: int value (default 15) for number of neighbors to consider for crossLID calculation in generated samples
        evaluation.
       random_seed: int value (default 145) for numpy random seeding when randomly organizing samples in the data that
        will be split into batches.
       n_generated_samples: int value (default 96) for number of samples generated to test the model during training.
    """
    
    # Obtaining the train data, randomize its order and divide it be twice the standard deviation of its values
    all_data = train_data_o.iloc[
        np.random.RandomState(seed=random_seed).permutation(len(train_data_o))]/(2*train_data_o.values.std())
    
    # Same treatment for the test data
    test_data = (test_data/(2*test_data.values.std())).values
    training_data = all_data
    train_data = all_data.values
    
    # Change class labels to numerical values while following the randomized ordered of samples
    if len(set(train_lbls)) < 3: # 1 and 0 for when there are only two classes
        train_labels = pd.get_dummies(
            np.array(train_lbls)[np.random.RandomState(seed=random_seed).permutation(len(train_data))]).values[:,0]
        test_labels = pd.get_dummies(np.array(test_lbls)).values[:,0]
    else: # One hot encoding for when there are more than two classes
        train_labels = pd.get_dummies(
            np.array(train_lbls)[np.random.RandomState(seed=random_seed).permutation(len(train_data))]).values
        test_labels = pd.get_dummies(np.array(test_lbls)).values
    # Save the order of the labels
    ordered_labels = pd.get_dummies(
            np.array(train_lbls)[np.random.RandomState(seed=random_seed).permutation(len(train_data_o))]).columns

    batch_divisions = int(batch_size / len(set(train_lbls))) # See how many samples of each class will be in each batch
    n_steps = epochs * int(training_data.shape[0] / batch_size) # Number of steps: nº of batches per epoch * nº of epochs
    n_critic = 5
    
    # Set up the evaluating images printed during training and the intervals they will be updated
    f, (axl, axc, axr) = plt.subplots(1, 3, figsize = (16,5))
    update1 = n_steps//200
    update2 = n_steps//20

    if hasattr(tqdm, '_instances'):
        tqdm._instances.clear() # clear if it exists

    i=0

    for step in tqdm(range(n_steps)):
        
        # Critic Training
        crit_loss_temp = []
        
        # Select real samples for this batch on training and order samples to put samples of the same class together
        real_samp = train_data[i*batch_size:(i+1)*batch_size]
        real_lbls = train_labels[i*batch_size:(i+1)*batch_size]

        real_samples = np.empty(real_samp.shape)
        real_labels = np.empty(real_lbls.shape)
        a = 0
        if len(set(train_lbls)) < 3:
            for l,s in sorted(zip(real_lbls, real_samp), key=lambda pair: pair[0], reverse=True):
                real_samples[a] = s
                real_labels[a] = l
                a = a+1
        else:
            for l,s in sorted(zip(real_lbls, real_samp), key=lambda pair: np.argmax(pair[0]), reverse=False):
                #print(l, np.argmax(l))
                real_samples[a] = s
                real_labels[a] = l
                a = a+1

        for _ in range(n_critic): # For each step, train critic n_critic times
            
            # Generate input for generator
            artificial_samples = tf.constant(input_dist.rvs(size=all_data.shape[1]*batch_size), shape=[
                batch_size,all_data.shape[1]])
            artificial_labels = real_labels.copy()

            # Generate artificial samples from the latent vector
            artificial_samples = generator([artificial_samples, artificial_labels], training=True)
            
            with tf.GradientTape() as crit_tape: # See the gradient for the critic

                # Get the logits for the generated samples
                X_artificial = critic([artificial_samples, artificial_labels], training=True)
                # Get the logits for the real samples
                X_true = critic([real_samples, real_labels], training=True)

                # Calculate the critic loss using the generated and real sample results
                c_cost = critic_loss_wgan(X_true, X_artificial)

                # Calculate the gradient penalty
                grad_pen = gradient_penalty_cwgan(batch_size, real_samples, artificial_samples,
                                                  real_labels, artificial_labels, critic)
                # Add the gradient penalty to the original discriminator loss
                crit_loss = c_cost + grad_pen * grad_pen_weight
                
            crit_loss_temp.append(crit_loss)

            # Calculate and apply the gradients obtained from the loss on the trainable variables
            gradients_of_critic = crit_tape.gradient(crit_loss, critic.trainable_variables)
            critic_optimizer.apply_gradients(zip(gradients_of_critic, critic.trainable_variables))

        i = i + 1
        if (step+1) % (n_steps//epochs) == 0:
            i=0

        crit_loss_all.append(np.mean(crit_loss_temp))
        
        # Generator Training
        # Generate inputs for generator, values and labels
        artificial_samples = tf.constant(input_dist.rvs(size=all_data.shape[1]*batch_size), shape=[
                batch_size,all_data.shape[1]])
        
        if len(set(train_lbls)) < 3:
            artificial_labels = tf.constant([1.0]*(batch_size//2) + [0.0]*(batch_size//2), shape=(batch_size,1))
        else:
            artificial_labels = np.array(pd.get_dummies([i for i in range(len(set(train_lbls)))]*batch_divisions))
    
        with tf.GradientTape() as gen_tape: # See the gradient for the generator
            # Generate artificial samples
            artificial_samples = generator([artificial_samples, artificial_labels], training=True)
            
            # Get the critic results for generated samples
            X_artificial = critic([artificial_samples, artificial_labels], training=True)
            # Calculate the generator loss
            gen_loss = generator_loss_wgan(X_artificial)

        # Calculate and apply the gradients obtained from the loss on the trainable variables
        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        gen_loss_all.append(gen_loss)

        # Update the progress bar and evaluation graphs every update1 steps for loss plots and update2 for the others.
        if (step + 1) % update1 == 0:
            
            # Update the evaluating figures at the set intervals
            axl.clear() # Always clear the corresponding ax before redrawing it
            
            # Loss Plot
            axl.plot(gen_loss_all, color = 'blue', label='Generator Loss')
            axl.plot(crit_loss_all,color = 'red', label='Critic Loss')
            axl.set_xlabel('Number of Steps')
            axl.set_ylabel('Loss')
            axl.legend()
            
            ipythondisplay.clear_output(wait=True)
            ipythondisplay.display(plt.gcf())

        if (step + 1) % update2 == 0:

            saved_predictions.append(generate_predictions(generator, n_generated_samples, all_data.shape[1], 
                                                          input_realdata_dist, ordered_labels))
            # See density and coverage and crossLID (divided by 25 to be in the same order as the rest) 
            # of latest predictions
            den, cov = gem.evaluation_coverage_density(test_data, saved_predictions[-1], k= k_cov_den, metric='euclidean')
            clid = gem.cross_LID_estimator_byMLE(test_data, saved_predictions[-1], k=k_crossLID, metric='euclidean')/25
            density.append(den)
            coverage.append(cov)
            crossLID.append(clid)

            # PCA of the latest predictions and training data
            # Divide by twice the standard deviation to be the same as the generated data
            dfs_temp = pd.concat((train_data_o/(2*train_data_o.values.std()),pd.DataFrame(
                saved_predictions[-1].numpy(), columns=train_data_o.columns))) 
            temp_lbls = train_lbls.copy()
            for l in ordered_labels:
                temp_lbls.extend([l+' - GAN']*(n_generated_samples//len(ordered_labels)))
            principaldf = gem.pca_sample_projection(dfs_temp, temp_lbls, pca, whiten=True, 
                                                samp_number=len(train_data_o.index))
            lcolors = label_colors_test

            # Hierarchical clustering of the latest predictions and testing data, 
            # saving the correct 1st cluster fraction results
            dfs_temp = np.concatenate((test_data, saved_predictions[-1].numpy()))
            temp_lbls = ['real']*len(test_data) + ['gen']*len(saved_predictions[-1])
            hca_results = gem.perform_HCA(dfs_temp, temp_lbls, metric='euclidean', method='ward')
            corr1stcluster.append(hca_results['correct 1st clustering'])
            
            # Plots
            axc.clear()
            axc.plot(range(update2, step+2, update2), coverage, label='coverage')
            axc.plot(range(update2, step+2, update2), density, label='density')
            axc.plot(range(update2, step+2, update2), crossLID, color='red', label='crossLID')
            axc.plot(range(update2, step+2, update2), corr1stcluster, color='purple', label='corr_cluster')
            axc.legend()

            axr.clear()
            gem.plot_PCA(principaldf, lcolors, components=(1,2), title='', ax=axr)
            axr.legend(loc='upper right', ncol=1, framealpha=1)
            
            ipythondisplay.clear_output(wait=True)
            ipythondisplay.display(plt.gcf())

### Training the GAN

In [ ]:
GENERATE=True

epochs = 600
batch_size = 32
k_cov_den = 20
k_crossLID = 15
random_seed = 145
n_generated_samples = 48*len(pd.unique(lbls))

if GENERATE:
    # Store results
    gen_loss_all = []
    crit_loss_all = []
    saved_predictions = []
    coverage = []
    density = []
    crossLID = []
    corr1stcluster = []
    
    df = data_treated
    pca = PCA(n_components=2, svd_solver='full', whiten=True)
    pc_coords = pca.fit_transform(df)

    generator_optimizer = tf.keras.optimizers.RMSprop(1e-4)
    critic_optimizer = tf.keras.optimizers.RMSprop(1e-4)

    generator = generator_model(data_treated.shape[1], data_treated.shape[1], 128, 2)
    critic = critic_model(data_treated.shape[1], 512, 2)

    training_montage(data_treated, lbls, real_samples_all, labels, epochs, generator, critic, generator_optimizer,
                     critic_optimizer,
                     input_realdata_dist, batch_size, grad_pen_weight=10, k_cov_den=k_cov_den, k_crossLID=k_crossLID,
                     random_seed=random_seed, n_generated_samples=n_generated_samples)

    results={'gen_loss': gen_loss_all, 'crit_loss': crit_loss_all, 'saved_pred': saved_predictions,
             'coverage': coverage, 'density': density, 'crossLID': crossLID, 'corr1st_cluster': corr1stcluster}

In [ ]:
# Save the models and results
import pickle
if GENERATE:
    # Store data (serialize)
    with open('gan_models/LGDint_results.pickle', 'wb') as handle:
        pickle.dump(results, handle)

    # Save the generator and critic models' weights.
    generator.save_weights('gan_models/LGDint_gen')
    critic.save_weights('gan_models/LGDint_crit')

# Read back the models
if not GENERATE:
    # Setting the model up
    generator_optimizer = tf.keras.optimizers.RMSprop(1e-4)
    critic_optimizer = tf.keras.optimizers.RMSprop(1e-4)

    generator = generator_model(data_treated.shape[1], data_treated.shape[1], 128, 2)
    critic = critic_model(data_treated.shape[1], 512, 2)

    # Load Previously saved models
    generator.load_weights('./gan_models/LGDint_gen')
    critic.load_weights('./gan_models/LGDint_crit')

    with open('gan_models/LGDint_results.pickle', 'rb') as handle:
        results = pickle.load(handle)
        
    gen_loss_all, crit_loss_all, saved_predictions = results['gen_loss'], results['crit_loss'], results['saved_pred']
    coverage, density, crossLID = results['coverage'], results['density'], results['crossLID']
    corr1stcluster = results['corr1st_cluster']

#### Generate examples from our new code


- Generate examples in bulk - predictions (GAN data)
- Select only the 5 most correlated generated samples with each of the original samples - corr_preds (CorrGAN Data)

#### CorrGAN

In [ ]:
num_examples_to_generate = 2048
# Get distribution of intensity values of the dataset
hist = np.histogram(real_samples_all.values.flatten(), bins=100)
input_realdata_dist = stats.rv_histogram(hist)

test_input = tf.constant(input_realdata_dist.rvs(size=data_treated.shape[1]*num_examples_to_generate), 
                         shape=[num_examples_to_generate, data_treated.shape[1]])
test_labels = tf.constant([1]*(num_examples_to_generate//2) + [0]*(num_examples_to_generate//2), shape=[
        num_examples_to_generate,1])

predictions = generator([test_input, test_labels], training=False)
predictions = pd.DataFrame(predictions.numpy(), columns=data_treated.columns) * 2*data_treated.values.std()
predictions

test_labels = tf.constant([1]*(num_examples_to_generate//2) + [0]*(num_examples_to_generate//2), shape=[
        num_examples_to_generate,])

See correlation between samples and choose the 5 most correlated generated samples for each of the original samples.

In [ ]:
df = real_samples_all
# Calculate all correlations between all samples of real and artificial data and store them in a dataframe
correlations = pd.DataFrame(index=predictions.index, columns=df.index).astype('float')

for i in df.index:
    for j in predictions.index:
        correlations.loc[j,i] = stats.pearsonr(df.loc[i],
                                               predictions.loc[j])[0]

In [ ]:
# Indices to keep in the correlated GAN data
idx_to_keep = []
for i in correlations:
    idx_to_keep.extend(correlations[i].sort_values(ascending=False).index[:5])
    
print('Nº of total idx :', len(idx_to_keep))
print('Nº of unique idx:', len(set(idx_to_keep)))

In [ ]:
# Make the correlation GAN dataframe and corresponding label targets
corr_preds = predictions.loc[list(set(idx_to_keep))]
corr_lbls  = list(np.array(test_labels)[list(set(idx_to_keep))])

#### GAN Data

In [ ]:
random_seed=145
ordered_labels = pd.get_dummies(
            np.array(lbls)[np.random.RandomState(seed=random_seed).permutation(len(lbls))]).columns
ordered_labels

In [ ]:
num_examples_to_generate = 1024
# Get distribution of intensity values of the dataset
hist = np.histogram(real_samples_all.values.flatten(), bins=100)
input_realdata_dist = stats.rv_histogram(hist)

test_input = tf.constant(input_realdata_dist.rvs(size=data_treated.shape[1]*num_examples_to_generate), 
                         shape=[num_examples_to_generate, data_treated.shape[1]])
test_labels = tf.constant([1]*(num_examples_to_generate//2) + [0]*(num_examples_to_generate//2), shape=[
        num_examples_to_generate,1])

predictions = generator([test_input, test_labels], training=False)
predictions = pd.DataFrame(predictions.numpy(), columns=data_treated.columns) 
predictions = predictions * 2*data_treated.values.std()

test_labels = tf.constant([1]*(num_examples_to_generate//2) + [0]*(num_examples_to_generate//2), shape=[
        num_examples_to_generate,])
last_preds_labels = ['Cold-phase']*(len(predictions)//2) + ['Reperfusion']*(len(predictions)//2)

Transforming the 96 generated samples for 20 different time points during GAN training (stored in results) into DataFrames.

In [ ]:
saved_predictions = []
# Transform predictions into Pandas DataFrames
for i in range(len(results['saved_pred'])):
    saved_predictions.append(pd.DataFrame(results['saved_pred'][i].numpy(), 
                                          columns=data_treated.columns)* 2*data_treated.values.std())

### Loss Plot and PCAs and tSNEs representation on the evolution of generated samples with epochs

Measures of progression of the model in time.

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        steps_per_epoch = int(data_treated.shape[0] / batch_size)
        f, ax = plt.subplots(1, 1, figsize=(4,4))
        ax.plot(range(1,len(gen_loss_all)+1), gen_loss_all, label='Generator Loss')
        ax.plot(range(1,len(crit_loss_all)+1), crit_loss_all, label='Critic Loss')
        ax.set_xticks(range(0, (epochs+1)*steps_per_epoch, 200*steps_per_epoch))
        ax.set_xticklabels(range(0, (epochs+1), 200))

        ax.legend(fontsize=12)
        ax.set_xlim([0*steps_per_epoch,(epochs)*steps_per_epoch])
        ax.set_xlabel('Nº of Epochs', fontsize=15)
        ax.set_ylabel('Loss', fontsize=15)
        ax.set_title('Loss Plot', fontsize=15)
        plt.tight_layout()
        f.savefig('images/LGDint_LossPlot.png' , dpi=400)

**PCA and tSNE of GAN generated data and the linearly generated 'test data'**

Progression with number of epochs.

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1):
        f, axs = plt.subplots(2,5, figsize=(16,8), constrained_layout=True)
        
        for i, ax in zip(range(0, len(saved_predictions),2), axs.ravel()):
            dfs_temp = pd.concat((data_treated, saved_predictions[i]))
            temp_lbls = lbls.copy()
            temp_lbls.extend(['Cold-phase - GAN']*(len(saved_predictions[-1])//2))
            temp_lbls.extend(['Reperfusion - GAN']*(len(saved_predictions[-1])//2))
            
            principaldf = ma.compute_df_with_PCs(dfs_temp, n_components=2, whiten=True, labels=temp_lbls,
                                                 return_var_ratios=False)

            lcolors = label_colors_test
            
            gem.plot_PCA(principaldf, lcolors, components=(1,2), title='', ax=ax)
            #gem.plot_ellipses_PCA(principaldf, lcolors, components=(1,2),ax=ax, q=0.95)
            ax.legend(loc='upper right', ncol=1, framealpha=1)

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1):
        f, axs = plt.subplots(2,5, figsize=(16,8), constrained_layout=True)
        
        for i, ax in zip(range(0, len(saved_predictions), 2), axs.ravel()):
            
            dfs_temp = pd.concat((data_treated, saved_predictions[i]))
            temp_lbls = lbls.copy()
            temp_lbls.extend(['Cold-phase - GAN']*(len(saved_predictions[-1])//2))
            temp_lbls.extend(['Reperfusion - GAN']*(len(saved_predictions[-1])//2))
            
            X = dfs_temp.copy()
            X_embedded = TSNE(n_components=2, perplexity=30, learning_rate='auto',
                              init='random', verbose=0).fit_transform(X)

            df = X_embedded
            labels = temp_lbls
            lcolors = label_colors_test
            
            gem.plot_tSNE(df, labels, lcolors, components=(1,2), title='', ax=ax)
            gem.plot_ellipses_tSNE(df, labels, lcolors, components=(1,2),ax=ax, q=0.95)
            ax.legend(loc='upper right', ncol=1, framealpha=1)

#### PCA of GAN generated data and experimental (real) data

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1):
        f, ax = plt.subplots(1,1, figsize=(4,4), constrained_layout=True)
            
        p = saved_predictions[-1].copy()
        p.columns = real_samples_all.columns
        dfs_temp = pd.concat((real_samples_all, p))
        temp_lbls = target.copy()
        temp_lbls.extend(['Cold-phase - GAN']*(len(saved_predictions[-1])//2))
        temp_lbls.extend(['Reperfusion - GAN']*(len(saved_predictions[-1])//2))

        principaldf = ma.compute_df_with_PCs(dfs_temp, n_components=2, whiten=True, labels=temp_lbls,
                                                 return_var_ratios=False)

        lcolors = label_colors_test

        gem.plot_PCA(principaldf, lcolors, components=(1,2), title='', ax=ax)
        gem.plot_ellipses_PCA(principaldf, lcolors, components=(1,2),ax=ax, q=0.95)
        ax.set_ylabel('PC 2', fontsize=15)
        ax.set_xlabel('PC 1', fontsize=15)
        #ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=14)
        f.savefig('images/LGDint_PCAPlot.png' , dpi=400)

### Comparing GAN Generated Data Characteristics with experimental data

In [ ]:
names = ['Experimental data', 'GAN data', 'CorrGAN data']
data_repo = [real_samples_all, predictions, corr_preds]
tgs = [target, last_preds_labels, corr_lbls]
data_characteristics = [gem.characterize_data(ds, name, tg) for ds,name,tg in zip(data_repo, names, tgs)]
data_characteristics = pd.DataFrame(data_characteristics).set_index('Dataset')
data_characteristics

In [ ]:
f, (axl, axr) = plt.subplots(1,2, figsize=(8,4))

names = ['Experimental', 'GAN', 'CorrGAN']
axl.boxplot([ds.values.flatten() for ds in data_repo])
axl.set_ylabel('Distribution of scaled intensities', fontsize=15)
axl.set_xticklabels(names, fontsize=12)
#axl.set_yticks([-2, 0, 2, 4])

axr.boxplot([ds.sum(axis=1) for ds in data_repo])
axr.set_ylabel('Distribution of feature sum of intensities', fontsize=12)
axr.set_xticklabels(names, fontsize=12)

plt.tight_layout()
plt.show()
f.savefig('images/LGDint_characteristics.png' , dpi=400)

### Hierarchical Clustering

Hierarchical clustering of 96 GAN samples and experimental data. 

In [ ]:
# Hierarchical clustering of 96 GAN samples and real data
dt = real_samples_all
dfs_temp = np.concatenate((dt, saved_predictions[-1].values))
test_labels = target.copy()
temp_lbls = list(test_labels)
for i in ['Cold-phase','Reperfusion']:
    temp_lbls.extend([i+' - GAN']*(n_generated_samples//2))

hca_results = gem.perform_HCA(dfs_temp, temp_lbls, metric='euclidean', method='ward')

In [ ]:
f, ax = plt.subplots(figsize=(3, 10))
gem.plot_dendogram(hca_results['Z'], 
               temp_lbls, ax=ax,
               label_colors=label_colors_test, title='',
               color_threshold=0)
ax.set_yticklabels([])
ax.set_xticks([])
plt.show()
f.savefig('images/LGDint_HCAPlot.png' , dpi=400)

### Coverage and Density

In [ ]:
training_set = data_treated

density_list, coverage_list = gem.evaluation_coverage_density_all_k_at_once(training_set, 
                                                                            predictions, 
                                                                            metric='Euclidean')

density_list_test, coverage_list_test = gem.evaluation_coverage_density_all_k_at_once(training_set,
                                                                                      real_samples_all, 
                                                                                      metric='Euclidean')

density_list_real, coverage_list_real = gem.evaluation_coverage_density_all_k_at_once(real_samples_all, 
                                                                                      predictions,
                                                                                      metric='Euclidean')

density_list_corr, coverage_list_corr = gem.evaluation_coverage_density_all_k_at_once(training_set, 
                                                                                      corr_preds,
                                                                                      metric='Euclidean')

density_list_linaug_real, coverage_list_linaug_real = gem.evaluation_coverage_density_all_k_at_once(
                                                                                      real_samples_all, 
                                                                                      training_set,
                                                                                      metric='Euclidean')

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, (axl, axr) = plt.subplots(1, 2, figsize = (8,4))#, sharey='row')#, sharex='col')
        
        axl.plot(range(1,len(training_set)), density_list, label='Density', color='blue')
        axl.plot(range(1,len(training_set)), coverage_list, label='Coverage', color='red')
        axl.set_title('GAN - Interpolated Data', fontsize=15)
        axl.set_ylabel('Density / Coverage', fontsize=15)
        axl.set_ylim([0,5])
        axl.set_xlabel('Nº of Nearest Neighbors', fontsize=15)

        axr.plot(range(1,len(real_samples_all)), density_list_real, label='Density', color='blue')
        axr.plot(range(1,len(real_samples_all)), coverage_list_real, label='Coverage', color='red')
        axr.set_title('GAN - Experimental Data', fontsize=15)
        axr.set_ylim([0,5])
        
        axr.legend()
        axl.legend()
        axr.set_xlabel('Nº of Nearest Neighbors', fontsize=15)

    plt.tight_layout()
    f.savefig('images/LGDint_DenCovPlot.png' , dpi=400)

### Histograms
 
Histograms of Values of normal Experimental (Real), Generated and GAN Generated Data.

In [ ]:
# Predictions
last_preds = predictions

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize = (6,4))#, sharey='row')#, sharex='col')
        X = np.arange(-8, 8.01, 0.05)
        hist = np.histogram(real_samples_all.values.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='blue', label='Experimental')
        
        hist = np.histogram(data_treated.values.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='orange', label='Interpolated')
        ax.set_ylabel('PDF', fontsize=15)
        
        hist = np.histogram(last_preds.values.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='red', label='GAN')
        
        hist = np.histogram(corr_preds.values.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='green', label='CorrGAN')
        
        ax.set_ylabel('PDF', fontsize=15)
        ax.set_xlabel('Scaled Intensities', fontsize=15)
        #ax.set_xticks([-8, -4, 0, 4, 8])
        ax.set_ylim([0,0.6])

        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=12, handlelength=1)
        ax.set_title('Intensities Distribution', fontsize=18)
        plt.tight_layout()
        f.savefig('images/LGDint_IntPlot.png', dpi=400)

### Correlations between samples of Experimental (Real) Data, Linearly Interpolated and GAN Generated Data

In [ ]:
correlation_real_real = np.corrcoef(real_samples_all)
print('Correlation Experimental-Experimental calculation ended.')

correlation_lin_lin = np.corrcoef(data_treated)
print('Correlation Linear-Linear calculation ended.')

correlation_gan_gan = np.corrcoef(last_preds)
print('Correlation GAN-GAN calculation ended.')

correlation_corr_corr = np.corrcoef(corr_preds)
print('Correlation CorrGAN-CorrGAN calculation ended.')

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize = (5,4))#, sharey='row')#, sharex='col')
        X = np.arange(-1, 1.01, 0.05)
        hist = np.histogram(correlation_real_real.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='blue', label='Experimental')
        ax.set_ylabel('PDF', fontsize=15)

        hist = np.histogram(correlation_lin_lin.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='orange', label='Interpolated')

        hist = np.histogram(correlation_gan_gan.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='red', label='GAN')

        hist = np.histogram(correlation_corr_corr.flatten(), bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='green', label='CorrGAN')

        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=12, handlelength=0.9, handletextpad=0.5)
        ax.set_title('Sample Correlations', fontsize=18)
        ax.set_xlabel('Correlation', fontsize=15)
        #ax.set_ylim([0,3.6])
    #f.text(0.5, 0.05, 'Correlation', ha='center', va='top', fontsize=15)
    plt.tight_layout()
    f.savefig('images/LGDint_SampCorrPlot.png', dpi=400)

### Correlations between features of Experimental (Real), Linearly Interpolated and GAN Generated Samples

In [ ]:
correlation_real_real = np.corrcoef(real_samples_all.T)
print('Correlation Experimental-Experimental calculation ended.')

correlation_gen_gen = np.corrcoef(data_treated.T)
print('Correlation Generated-Generated calculation ended.')

correlation_gan_gan = np.corrcoef(last_preds.T)
print('Correlation GAN-GAN calculation ended.')

correlation_corr_corr = np.corrcoef(corr_preds.T)
print('Correlation CorrGAN-CorrGAN calculation ended.')

In [ ]:
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.2):
        f, ax = plt.subplots(1, 1, figsize = (5,4))#, sharey='row')#, sharex='col')
        X = np.arange(-1, 1.01, 0.05)
        hist = np.histogram(correlation_real_real.flatten()[~np.isnan(correlation_real_real.flatten())],
                            bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='blue', label='Experimental')
        ax.set_ylabel('PDF', fontsize=15)
        
        hist = np.histogram(correlation_gen_gen.flatten()[~np.isnan(correlation_gen_gen.flatten())], bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='orange', label='Interpolated')
        
        hist = np.histogram(correlation_gan_gan.flatten()[~np.isnan(correlation_gan_gan.flatten())], bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='red', label='GAN')
        
        hist = np.histogram(correlation_corr_corr.flatten()[~np.isnan(correlation_corr_corr.flatten())],
                            bins=50)
        hist_dist = stats.rv_histogram(hist)
        ax.plot(X, hist_dist.pdf(X), color='green', label='CorrGAN')
        
        
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=12, handlelength=0.8, handletextpad=0.4)
        ax.set_title('Feature Correlations', fontsize=18)
        ax.set_xlabel('Correlation', fontsize=15)
        ax.set_ylim([0,2])

    #f.text(0.5, 0.05, 'Correlation', ha='center', va='top', fontsize=15)
    plt.tight_layout()
    f.savefig('images/LGDint_FeatCorrPlot.png', dpi=400)

### Sample Correlation Matrix

Between samples of the experimental (real) data and a set of generated GAN samples.

In [ ]:
# Experimental (Real) Data, organize it to have first all samples of a class, then all samples of the other class
df = real_samples_all.copy()

samp = df.index
tg = target.copy()
new_order = [x for _, x in sorted(zip(tg, samp))]
new_tg = [x for x, _ in sorted(zip(tg, samp))]

df = df.loc[new_order]
#df

In [ ]:
# Calculate all correlations between all samples of experimental (real) and generated data and store them in a dataframe
correlations = pd.DataFrame(index=last_preds.index, columns=df.index).astype('float')

for i in df.index:
    for j in last_preds.index:
        correlations.loc[j,i] = stats.spearmanr(df.loc[i],
                                               last_preds.loc[j])[0]

correlations.columns = new_tg
correlations.index = ['Cold-phase - GAN']*(len(last_preds)//2) + ['Reperfusion - GAN']*(len(last_preds)//2)

In [ ]:
# Draw the clustermap

row_cols = [label_colors_test[lbl] for lbl in new_tg]
row_cols2 = [label_colors_test[lbl] for lbl in correlations.index]
co = sns.diverging_palette(160, 310, s=90, as_cmap=True, l=40)
g = sns.clustermap(correlations, col_colors=row_cols, cmap=co, row_colors= row_cols2, vmin=-1, vmax=1, method='ward',
                  cbar_pos = (-0.15, 0.1, 0.05, 0.5))
g.fig.set_size_inches((6,9))
# some tweaks
patches = []
for lbl in ordered_labels_test:
    patches.append(mpatches.Patch(color=label_colors_test[lbl], label=lbl))
leg = plt.legend(handles=patches, loc=3, bbox_to_anchor=(-1.35, 1.35, 0.5, 1),
                     frameon=False, fontsize=14) 
g.ax_heatmap.set_ylabel('GAN Data', fontsize=20)
g.ax_heatmap.set_xlabel('Experimental Data', fontsize=20)
g.ax_heatmap.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False,
                         labelright=False)

# Manually specify colorbar labelling after it's been generated
colorbar = g.ax_heatmap.collections[0].colorbar
colorbar.ax.tick_params(labelsize=13) 
plt.text(1, 1.10, 'Correlations', fontsize=14, horizontalalignment='center')
plt.show()
g.savefig('images/LGDint_Clustermap.png', dpi=400)

In [ ]:
a = scipy.spatial.distance_matrix(last_preds, df)
sample_distances = pd.DataFrame(a, index=last_preds.index, columns=df.index).astype('float')
sample_distances.index = ['Cold-phase - GAN']*(len(last_preds)//2) + ['Reperfusion - GAN']*(len(last_preds)//2)

In [ ]:
# Draw the clustermap
row_cols = [label_colors_test[lbl] for lbl in new_tg]
row_cols2 = [label_colors_test[lbl] for lbl in sample_distances.index]
g = sns.clustermap(sample_distances, col_colors=row_cols, cmap='bwr', row_colors= row_cols2,
                   #norm=matplotlib.colors.LogNorm(vmin=25, vmax=175),
                   vmin=20, vmax=100,
                   method='ward',
                  cbar_pos = (-0.13, 0.1, 0.05, 0.5))
g.fig.set_size_inches((6,9))
# some tweaks
patches = []
for lbl in ordered_labels_test:
    patches.append(mpatches.Patch(color=label_colors_test[lbl], label=lbl))
leg = plt.legend(handles=patches, loc=3, bbox_to_anchor=(-2.2, 1.45, 0.5, 1),
                     frameon=False, fontsize=14) 
g.ax_heatmap.set_ylabel('GAN Data', fontsize=20)
g.ax_heatmap.set_xlabel('Experimental Data', fontsize=20)
g.ax_heatmap.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False,
                         labelright=False)

# Manually specify colorbar labelling after it's been generated
colorbar = g.ax_heatmap.collections[0].colorbar
colorbar.ax.tick_params(labelsize=13) 
plt.text(1, 105.10, 'Distances', fontsize=14, horizontalalignment='center')
plt.show()

In [ ]:
sample_distances.max()

In [ ]:
sample_distances.min()

In [ ]:
# Experimental (Real) Data, organize it to have first all samples of a class, then all samples of the other class
df = real_samples_all.copy()

samp = df.index
tg = target.copy()
new_order = [x for _, x in sorted(zip(tg, samp))]
new_tg = [x for x, _ in sorted(zip(tg, samp))]

df = df.loc[new_order]
#df
a = scipy.spatial.distance_matrix(df, df)
sample_distances = pd.DataFrame(a, index=df.index, columns=df.index).astype('float')

In [ ]:
sample_distances.max()

In [ ]:
# Draw the clustermap

row_cols = [label_colors_test[lbl] for lbl in new_tg]
row_cols2 = [label_colors_test[lbl] for lbl in new_tg]
g = sns.clustermap(sample_distances, col_colors=row_cols, cmap='bwr', row_colors= row_cols2,
                   #norm=matplotlib.colors.LogNorm(vmin=30, vmax=250),
                   vmin=20, vmax=100,
                   method='ward',
                  cbar_pos = (-0.13, 0.1, 0.05, 0.5))
g.fig.set_size_inches((6,9))
# some tweaks
patches = []
for lbl in ordered_labels_test:
    patches.append(mpatches.Patch(color=label_colors_test[lbl], label=lbl))
leg = plt.legend(handles=patches, loc=3, bbox_to_anchor=(-1.2, 1.35, 0.5, 1),
                     frameon=False, fontsize=14) 
g.ax_heatmap.set_ylabel('Experimental Data', fontsize=20)
g.ax_heatmap.set_xlabel('Experimental Data', fontsize=20)
g.ax_heatmap.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, 
                         labelright=False)

# Manually specify colorbar labelling after it's been generated
colorbar = g.ax_heatmap.collections[0].colorbar
colorbar.ax.tick_params(labelsize=13) 
plt.text(1, 110.10, 'Distances', fontsize=14, horizontalalignment='center')
plt.show()